# Fingerprints

In [1]:
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
import csv
with open('data/CHEMBL27-chembl_27_activity-Or8O5vK6yNZ1L5id7ev7PGK1lWhfDfRKRJbbuXPM9Os=.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=";")
    cnr1_ligands = [Chem.MolFromSmiles(m['Smiles']) for m in reader]

suppl = Chem.SDMolSupplier('../data/drugbank.sdf')
drugs = [m for m in suppl if m]

RDKit ERROR: [17:50:34] Explicit valence for atom # 7 N, 4, is greater than permitted
RDKit ERROR: [17:50:34] ERROR: Could not sanitize molecule ending on line 20009
RDKit WARNING: [17:50:35] Warning: conflicting stereochemistry at atom 5 ignored.
RDKit WARNING: [17:50:35] Warning: conflicting stereochemistry at atom 5 ignored.
RDKit ERROR: [17:50:34] ERROR: Explicit valence for atom # 7 N, 4, is greater than permitted
RDKit ERROR: [17:50:35] Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 6 7 8 9 10
RDKit ERROR: 
RDKit ERROR: [17:50:35] ERROR: Could not sanitize molecule ending on line 250947
RDKit ERROR: [17:50:35] ERROR: Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 6 7 8 9 10
RDKit ERROR: 
RDKit ERROR: [17:50:35] Explicit valence for atom # 17 O, 3, is greater than permitted
RDKit ERROR: [17:50:35] ERROR: Could not sanitize molecule ending on line 258130
RDKit ERROR: [17:50:35] ERROR: Explicit valence for atom # 17 O, 3, is greater than permitted
RDKit ERROR: [17:50:35] Can't kek

RDKit ERROR: [17:50:36] ERROR: Can't kekulize mol.  Unkekulized atoms: 16 17 18 19 20
RDKit ERROR: 
RDKit ERROR: [17:50:36] Can't kekulize mol.  Unkekulized atoms: 7 8 9 10 17
RDKit ERROR: 
RDKit ERROR: [17:50:36] ERROR: Could not sanitize molecule ending on line 468852
RDKit ERROR: [17:50:36] ERROR: Can't kekulize mol.  Unkekulized atoms: 7 8 9 10 17
RDKit ERROR: 
RDKit ERROR: [17:50:36] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7 8 9 10 11
RDKit ERROR: 
RDKit ERROR: [17:50:36] ERROR: Could not sanitize molecule ending on line 469201
RDKit ERROR: [17:50:36] ERROR: Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7 8 9 10 11
RDKit ERROR: 
RDKit ERROR: [17:50:36] Explicit valence for atom # 53 N, 4, is greater than permitted
RDKit ERROR: [17:50:36] ERROR: Could not sanitize molecule ending on line 472285
RDKit ERROR: [17:50:36] ERROR: Explicit valence for atom # 53 N, 4, is greater than permitted
RDKit ERROR: [17:50:36] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 12
RDKit ERROR

RDKit ERROR: [17:50:37] ERROR: Can't kekulize mol.  Unkekulized atoms: 0 1 2 4 5 6 7 8 9 10
RDKit ERROR: 
RDKit ERROR: [17:50:38] Explicit valence for atom # 0 Be, 3, is greater than permitted
RDKit ERROR: [17:50:38] ERROR: Could not sanitize molecule ending on line 995692
RDKit ERROR: [17:50:38] ERROR: Explicit valence for atom # 0 Be, 3, is greater than permitted
RDKit ERROR: [17:50:38] Explicit valence for atom # 0 C greater than permitted
RDKit ERROR: [17:50:38] ERROR: Could not sanitize molecule ending on line 1151592


In [2]:
cnr1_ligands_ecfp = [Chem.GetMorganFingerprintAsBitVect(m, 2, nBits=1024) for m in cnr1_ligands]
drugs_ecfp = [Chem.GetMorganFingerprintAsBitVect(m, 2, nBits=1024) for m in drugs]

In [11]:
def ecfp_sums_scale(sett):
    set_sums = [0]*sett[0].GetNumBits() 
    for key in sett:
        for onbit in key.GetOnBits():
            set_sums[onbit] += 1
    return [x/len(sett) for x in set_sums]

cnr1_ligands_ecfp_scaled = ecfp_sums_scale(cnr1_ligands_ecfp)
drugs_ecfp_scaled = ecfp_sums_scale(drugs_ecfp)

cnr1_drugbank_differences = [(i, a_b[0] - a_b[1]) for i, a_b in enumerate(zip(cnr1_ligands_ecfp_scaled, drugs_ecfp_scaled))]
cnr1_drugbank_differences.sort(key=lambda x: x[1])
cnr1_drugbank_differences[:3], cnr1_drugbank_differences[-3:]


([(147, -0.1908649392414332),
  (389, -0.14072527389841652),
  (1, -0.12348442285640562)],
 [(216, 0.31598216944157936),
  (659, 0.3279967342810963),
  (561, 0.3323803803828841)])

The most uncomon bit among MTOR is 147 and the most prevalent is 561, lets have a closer look that one. 

In [24]:
substructure_smiles = {}
for ligand in cnr1_ligands:
    for submol in substructures_of_bit_ecfp4_1024(ligand, 561):
        subsmiles = Chem.MolToSmiles(submol)
        substructure_smiles[subsmiles] = substructure_smiles.get(subsmiles, 0) + 1
substructure_smiles

{'CC=O': 188,
 'CCC': 441,
 'ccC': 399,
 'O=CO': 10,
 'CCN': 183,
 'Cco': 1,
 'NC=O': 326,
 'COc': 51,
 'N#CN': 6,
 'CCO': 53,
 'N=CN': 27,
 'CNC': 90,
 'O=S=O': 13,
 'cC#N': 116,
 'CCc': 60,
 'ccCl': 29,
 'CC#N': 115,
 'Cc[nH]': 4,
 'N[SH]=O': 4,
 'Ccn': 22,
 'ccO': 10,
 'CSc': 1,
 'COC': 33,
 'cnC': 48,
 'C=CC': 5,
 '[nH]c=O': 2,
 'ncN': 8,
 'CCS': 1,
 'CC=N': 17,
 'CNN': 3,
 'cC=O': 13,
 'ncO': 3,
 'CNc': 9,
 'NC=S': 4,
 'ccN': 4,
 'ccc': 1,
 'CCn': 3,
 'CC#C': 1,
 'C=CCl': 2,
 'FCF': 2,
 'CSC': 1,
 'C[SH]=O': 7,
 'cc=O': 6,
 'nc=O': 3,
 '[2H]C[2H]': 1}

The most frequent one is CCC, then ccC and CC=O. In Drugbank for that bit there is not so much structures, the most freaquent structure is aromatic ccc, that appears in CNR1 only ones. 

In [26]:
substructure_smiles_db = {}
for drug in drugs:
    for submol in substructures_of_bit_ecfp4_1024(drug, 561):
        subsmiles = Chem.MolToSmiles(submol)
        substructure_smiles_db[subsmiles] = substructure_smiles_db.get(subsmiles, 0) + 1
substructure_smiles_db

{'ccc': 164,
 'C=O.CO': 3,
 'CO.cCl': 36,
 'cCl.cCl': 102,
 'cCl.cn': 1,
 'cCl.cs': 5,
 'CCl.cCl': 1,
 'C=O.CN': 2,
 'cCl.ns': 2,
 'Cn.cCl': 1,
 'C=N.cc': 2,
 'CS.cCl': 4,
 'O=S.cCl': 19,
 'cCl.cF': 9,
 'cCl.cO': 9,
 'cSc': 23,
 'C=O.cCl': 14,
 'CN.cCl': 14,
 'CCN': 20,
 'cCl.co': 2,
 'C=N.cN': 1,
 'CF.cCl': 4,
 'c-n.cCl': 3,
 'cCl.cS': 9,
 'CO.cF': 1,
 'CC.CN': 1,
 'ccCl': 95,
 'ccO': 12,
 'ccC': 18,
 'CC.Cc': 1,
 'cc.cc': 7,
 'ClCCl': 3,
 'C=O.cO': 1,
 'c=O.cCl': 5,
 'CO.cO': 2,
 'CI.cCl': 3,
 'cOC': 2,
 'CNC': 23,
 'CC.cc': 10,
 'c-cn': 2,
 'cNc': 4,
 'ncN': 7,
 'CCC': 57,
 'COC': 7,
 'CSC': 2,
 'C=O.cN': 1,
 'CCl.CO': 1,
 '[NH3+][O-].cCl': 1,
 'C=S.cCl': 3,
 'O=S=O': 14,
 'O=CO': 27,
 'ccF': 8,
 'CNO': 1,
 'CCO': 17,
 'COc': 16,
 'ccN': 4,
 'CC=O': 12,
 'CSc': 3,
 'cCO': 8,
 'cnc': 14,
 'coc': 5,
 'ccn': 13,
 'n[nH]n': 1,
 '[O-][ClH2+3][O-]': 1,
 'Cnn': 1,
 'csc': 1,
 'OPO': 1,
 'cCC': 4,
 'CC.CC': 5,
 'CC[nH3+]': 1,
 'cCN': 3,
 'N=CN': 5,
 'C[NH2+]C': 2,
 'cCl.cc': 1,
 'CO.CO': 1,